In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 
%load_ext tensorboard

In [2]:
import tensorflow as tf 
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sklearn
import tensorflow.keras.backend as kb
import yaml
import datetime
import glob

In [3]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [4]:
from HelperFn.ExpData_Generator import TrainingGenerator
from HelperFn.ExpData_Generator import EvalTestGenerator
from HelperFn.utils import Params
from sklearn.metrics import r2_score

In [1380]:
# recommended params.json is included in run_1 folder, ready for demo run. 
# archive_final_run folder contain pre-trained model (best model)
mini_EXP_name = 'new_all_combined/architecture4/batch_size128/runs2/run2.19.1'
exp_folder_path = 'experiments/' + mini_EXP_name

DATA_DIR = '../processed_input_data_all/'
NPY_FOLDER = 'slice_npy/'
ML_EXP = 'split_master/'
Train_Master = 'train_master.txt'
Eval_Master = 'eval_master.txt'
Test_Master = 'test_master.txt'
json_path = os.path.join(exp_folder_path, 'params.json')

In [1381]:
params = Params(json_path)

#input_slice_shape = (256,128,1)
input_slice_shape = (128,64,1)
batch_size = 128 #params.batch_size
MMT = 0.80 #params.bn_momentum
LR = 1e-3 #params.learning_rate
num_channels = params.num_channels
EPOCH = params.num_epochs

In [1382]:
## define model
model = Sequential()
model.add(layers.Input(shape=input_slice_shape))
channels = [num_channels, num_channels * 2]
for c in channels:
    model.add(layers.Conv2D(c, 5, padding='same'))
    model.add(layers.Conv2D(c, 5, dilation_rate = 2, padding='same'))
    model.add(layers.BatchNormalization(momentum=MMT))
    model.add(layers.Activation(activation='relu'))
    model.add(layers.MaxPooling2D(2))
model.add(layers.Flatten())
model.add(layers.Dense(channels[-1]))
model.add(layers.Dropout(0.1))
model.add(layers.BatchNormalization(momentum=MMT))
model.add(layers.Dense(1, activation='relu'))
#summarize the model
model.summary()

Model: "sequential_60"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_240 (Conv2D)          (None, 128, 64, 4)        104       
_________________________________________________________________
conv2d_241 (Conv2D)          (None, 128, 64, 4)        404       
_________________________________________________________________
batch_normalization_180 (Bat (None, 128, 64, 4)        16        
_________________________________________________________________
activation_120 (Activation)  (None, 128, 64, 4)        0         
_________________________________________________________________
max_pooling2d_120 (MaxPoolin (None, 64, 32, 4)         0         
_________________________________________________________________
conv2d_242 (Conv2D)          (None, 64, 32, 8)         808       
_________________________________________________________________
conv2d_243 (Conv2D)          (None, 64, 32, 8)       

In [1383]:
def MSE_wSD_loss(Y_true, KE_pred):
    KE_true = Y_true[:, 0]
    SD_true = Y_true[:, 1]
    mse = tf.keras.losses.MeanSquaredError()
    return mse(Y_true[:, 0], KE_pred)/((Y_true[:, 1])**1+1e-5)

def custom_accuracy_2SD(Y_true, KE_pred):
    KE_true = Y_true[:, 0]
    SD_true = Y_true[:, 1]
    KE_pred = KE_pred[:, 0]
    C_accuracy = kb.mean(kb.cast(kb.less_equal(kb.abs(KE_pred-KE_true),1*SD_true), dtype = "float32"))
    return C_accuracy 

In [1384]:
# compile the model
opt = tf.keras.optimizers.Adam(learning_rate=LR)
model.compile(optimizer=opt, loss = MSE_wSD_loss, 
              metrics=['mse', custom_accuracy_2SD])

In [1385]:
filepath = exp_folder_path + "/weights-improvement-{epoch:02d}-{val_custom_accuracy_2SD:.2f}.hdf5"
callbacks = [
#     tf.keras.callbacks.TensorBoard(log_dir='logs'),
     tf.keras.callbacks.EarlyStopping(monitor='loss',min_delta=0.001,
                                      patience=5,mode='min'),
    tf.keras.callbacks.ModelCheckpoint(
        filepath, monitor ='val_loss', verbose=1,
        save_best_only=True, mode= 'min')
]

In [1386]:
training_generator = TrainingGenerator(ML_EXP, NPY_FOLDER, Train_Master, batch_size = batch_size, data_dir=DATA_DIR)
validation_generator = EvalTestGenerator(ML_EXP, NPY_FOLDER, Eval_Master, batch_size = batch_size, data_dir=DATA_DIR)

18315
18315
 shuffling
3637
3637


In [1387]:
# configure early stopping
# fit the model
history = model.fit(x=training_generator, epochs=EPOCH,
                    validation_data=validation_generator, 
                    callbacks = callbacks,
                    use_multiprocessing=False)

Epoch 1/50
143/143 [==============================] - ETA: 0s - batch: 71.0000 - size: 128.0000 - loss: 1.7167 - mse: 0.2119 - custom_accuracy_2SD: 0.2543
Epoch 00001: val_loss improved from inf to 0.48257, saving model to experiments/new_all_combined/architecture4/batch_size128/runs2/run2.19.1/weights-improvement-01-0.48.hdf5
143/143 [==============================] - 151s 893ms/step - batch: 71.0000 - size: 128.0000 - loss: 1.7167 - mse: 0.2119 - custom_accuracy_2SD: 0.2543 - val_loss: 0.4826 - val_mse: 0.1070 - val_custom_accuracy_2SD: 0.4841
Epoch 2/50
143/143 [==============================] - ETA: 0s - batch: 71.0000 - size: 128.0000 - loss: 0.9394 - mse: 0.1665 - custom_accuracy_2SD: 0.3842
Epoch 00002: val_loss improved from 0.48257 to 0.29412, saving model to experiments/new_all_combined/architecture4/batch_size128/runs2/run2.19.1/weights-improvement-02-0.68.hdf5
143/143 [==============================] - 94s 655ms/step - batch: 71.0000 - size: 128.0000 - loss: 0.9394 - mse: 0

143/143 [==============================] - ETA: 0s - batch: 71.0000 - size: 128.0000 - loss: 0.2480 - mse: 0.1329 - custom_accuracy_2SD: 0.6780
Epoch 00014: val_loss did not improve from 0.21933
143/143 [==============================] - 124s 867ms/step - batch: 71.0000 - size: 128.0000 - loss: 0.2480 - mse: 0.1329 - custom_accuracy_2SD: 0.6780 - val_loss: 0.2235 - val_mse: 0.1173 - val_custom_accuracy_2SD: 0.7363
Epoch 15/50
143/143 [==============================] - ETA: 0s - batch: 71.0000 - size: 128.0000 - loss: 0.2268 - mse: 0.1329 - custom_accuracy_2SD: 0.7063
Epoch 00015: val_loss improved from 0.21933 to 0.21394, saving model to experiments/new_all_combined/architecture4/batch_size128/runs2/run2.19.1/weights-improvement-15-0.75.hdf5
143/143 [==============================] - 124s 869ms/step - batch: 71.0000 - size: 128.0000 - loss: 0.2268 - mse: 0.1329 - custom_accuracy_2SD: 0.7063 - val_loss: 0.2139 - val_mse: 0.1412 - val_custom_accuracy_2SD: 0.7542
Epoch 16/50
143/143 [====

143/143 [==============================] - ETA: 0s - batch: 71.0000 - size: 128.0000 - loss: 0.1019 - mse: 0.1268 - custom_accuracy_2SD: 0.7994
Epoch 00026: val_loss did not improve from 0.07030
143/143 [==============================] - 124s 867ms/step - batch: 71.0000 - size: 128.0000 - loss: 0.1019 - mse: 0.1268 - custom_accuracy_2SD: 0.7994 - val_loss: 0.1026 - val_mse: 0.0975 - val_custom_accuracy_2SD: 0.7369
Epoch 27/50
143/143 [==============================] - ETA: 0s - batch: 71.0000 - size: 128.0000 - loss: 0.0967 - mse: 0.1265 - custom_accuracy_2SD: 0.8026
Epoch 00027: val_loss did not improve from 0.07030
143/143 [==============================] - 124s 868ms/step - batch: 71.0000 - size: 128.0000 - loss: 0.0967 - mse: 0.1265 - custom_accuracy_2SD: 0.8026 - val_loss: 0.1826 - val_mse: 0.0826 - val_custom_accuracy_2SD: 0.5723
Epoch 28/50
143/143 [==============================] - ETA: 0s - batch: 71.0000 - size: 128.0000 - loss: 0.0967 - mse: 0.1266 - custom_accuracy_2SD: 0.8

143/143 [==============================] - ETA: 0s - batch: 71.0000 - size: 128.0000 - loss: 0.0793 - mse: 0.1263 - custom_accuracy_2SD: 0.8285
Epoch 00040: val_loss did not improve from 0.06651
143/143 [==============================] - 92s 645ms/step - batch: 71.0000 - size: 128.0000 - loss: 0.0793 - mse: 0.1263 - custom_accuracy_2SD: 0.8285 - val_loss: 0.1169 - val_mse: 0.1618 - val_custom_accuracy_2SD: 0.7427
Epoch 41/50
143/143 [==============================] - ETA: 0s - batch: 71.0000 - size: 128.0000 - loss: 0.0799 - mse: 0.1258 - custom_accuracy_2SD: 0.8297
Epoch 00041: val_loss did not improve from 0.06651
143/143 [==============================] - 88s 617ms/step - batch: 71.0000 - size: 128.0000 - loss: 0.0799 - mse: 0.1258 - custom_accuracy_2SD: 0.8297 - val_loss: 0.0954 - val_mse: 0.0992 - val_custom_accuracy_2SD: 0.7690
Epoch 42/50
143/143 [==============================] - ETA: 0s - batch: 71.0000 - size: 128.0000 - loss: 0.0784 - mse: 0.1257 - custom_accuracy_2SD: 0.833

In [1357]:
history.__dict__


{'validation_data': None,
 'model': <tensorflow.python.keras.engine.sequential.Sequential at 0x7f85e3f25d00>,
 '_chief_worker_only': None,
 '_supports_tf_logs': False,
 'history': {'loss': [1.2632883670446755,
   0.6587991831185934,
   0.4832261473565668,
   0.36673918939553773,
   0.3223308594702007,
   0.3086974420747557,
   0.2961585445629133,
   0.27913851264890255,
   0.232475399033173,
   0.23670432294582153,
   0.2297896044654446,
   0.21598084083387067,
   0.1920564142348883,
   0.17054788836024024,
   0.1578606401498501,
   0.14441970089724013,
   0.1365754682492543,
   0.12273261461641405,
   0.11818387948132895,
   0.11028417152958317,
   0.10279624711175064,
   0.0997334605121946,
   0.09454993807248302,
   0.09219897574894911,
   0.08981286966196307,
   0.08649674352336596,
   0.08467589157236206,
   0.08272108644038648,
   0.08192572049744479,
   0.08139683379159941,
   0.08105104815188821,
   0.07999431519033191,
   0.07952145489675182,
   0.07952354981244861,
   0.07862

In [1358]:
raw_history = history.history
with open (exp_folder_path + "/raw_history.yaml", "w") as filehandle:
    yaml.dump(history.history,filehandle)

In [1359]:
with open(exp_folder_path + "/raw_history.yaml", "r") as filehandle2:
    raw_history = yaml.load(filehandle2, Loader = yaml.Loader)

In [1360]:
val_loss = raw_history['val_loss']
train_accuracy = raw_history['custom_accuracy_2SD']
val_accuracy = raw_history['val_custom_accuracy_2SD']
best_model_EPOCH = val_loss.index(min(val_loss)) + 1 
selected_history = {}
selected_history['Bmodel_EPOCH'] = best_model_EPOCH
selected_history['min_val_loss'] =  np.round(min(val_loss),4)
selected_history['max_train_accuracy'] =  np.round(max(train_accuracy),4)
selected_history['max_val_accuracy'] =  np.round(max(val_accuracy),4)
selected_history['Bmodel_train_accuracy'] =  np.round(train_accuracy[val_loss.index(min(val_loss))],4)
selected_history['Bmodel_val_accuracy'] = np.round(val_accuracy[val_loss.index(min(val_loss))],4)

In [1361]:
selected_history

{'Bmodel_EPOCH': 37,
 'min_val_loss': 0.0658,
 'max_train_accuracy': 0.8387,
 'max_val_accuracy': 0.8733,
 'Bmodel_train_accuracy': 0.8288,
 'Bmodel_val_accuracy': 0.8733}

In [1362]:
plt.plot(raw_history['loss'][0:EPOCH])
plt.plot(raw_history['val_loss'][0:EPOCH])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.text(0.5, 0.5, mini_EXP_name, 
         horizontalalignment = 'center', transform=plt.gca().transAxes)
plt.legend(['Train','Eval'], loc='upper left')
plt.savefig(exp_folder_path  +'/loss_.png')
plt.show()

In [1363]:
plt.plot(raw_history['custom_accuracy_2SD'][0:EPOCH])
plt.plot(raw_history['val_custom_accuracy_2SD'][0:EPOCH])
plt.title('Model accuracy')
plt.ylabel('accuracy_2SD')
plt.xlabel('Epoch')
plt.text(0.5, 0.5, mini_EXP_name, 
         horizontalalignment = 'center', transform=plt.gca().transAxes)
plt.legend(['Train','Eval'], loc='upper left')
plt.savefig(exp_folder_path + '/custom_accuracy_2SD.png')
plt.show()

In [1364]:
files = glob.glob(exp_folder_path + '/*.hdf5',  
                   recursive = True)
saved_model = []
for file in files:
    print(file.split('/')[-1])
    saved_model.append(file.split('/')[-1])

weights-improvement-21-0.86.hdf5
weights-improvement-13-0.79.hdf5
weights-improvement-12-0.74.hdf5
weights-improvement-03-0.76.hdf5
weights-improvement-11-0.77.hdf5
weights-improvement-14-0.82.hdf5
weights-improvement-37-0.87.hdf5
weights-improvement-01-0.57.hdf5
weights-improvement-02-0.55.hdf5
weights-improvement-20-0.83.hdf5


In [1365]:
# Every epoch that show improving performance will be saved. 
# weights-improvment-'Epoch'-'Eval_2SD_Accuracy'
# 'selectedE' must be updated as 'Epoch' in which 'Eval_2SD_Accuracy' is highest 

selectedE = '37'

for file in saved_model:
    if file.split('-')[-2][0:2]==str(selectedE):
        selected_weights = file
print (selected_weights)  

weights-improvement-37-0.87.hdf5


In [1366]:
filepath = exp_folder_path + '/' + selected_weights 
print(filepath)

experiments/new_all_combined/architecture4/batch_size128/runs2/run2.24.5/weights-improvement-37-0.87.hdf5


In [1367]:
best_model = tf.keras.models.load_model(
    filepath, custom_objects = {"MSE_wSD_loss":MSE_wSD_loss, 
                                "custom_accuracy_2SD": custom_accuracy_2SD})
# best_model = tf.keras.models.load_model(filepath)
# best_model = tf.keras.models.load_model(filepath, custom_objects={"MSE_wSD_loss":MSE_wSD_loss})
# best_model = tf.keras.models.load_model(filepath)

In [1368]:
#apply best model to three subset of data

##Training
train_generator = EvalTestGenerator(ML_EXP, NPY_FOLDER, Train_Master, 
                                       batch_size = batch_size, 
                                       data_dir=DATA_DIR, shuffle = False)
train_label = np.hstack([batch[1][:,0] for batch in train_generator])
train_SD = np.hstack([batch[1][:,1] for batch in train_generator])
train_pred = best_model.predict(x=train_generator)

##Validation
eval_generator = EvalTestGenerator(ML_EXP, NPY_FOLDER, Eval_Master, 
                                         batch_size = batch_size, 
                                         data_dir=DATA_DIR, shuffle = False)
eval_label = np.hstack([batch[1][:,0] for batch in eval_generator])
eval_SD = np.hstack([batch[1][:,1] for batch in eval_generator])
eval_pred = best_model.predict(x=eval_generator)

##Testing
test_generator = EvalTestGenerator(ML_EXP, NPY_FOLDER, Test_Master, 
                                   batch_size = batch_size, 
                                   data_dir=DATA_DIR, shuffle = False)
test_label = np.hstack([batch[1][:,0] for batch in test_generator])
test_SD = np.hstack([batch[1][:,1] for batch in test_generator])
test_pred = best_model.predict(x=test_generator)

18315
18315
3637
3637
2204
2204


In [1369]:
train_LSP = [train_label,train_SD,train_pred]
eval_LSP = [eval_label,eval_SD,eval_pred]
test_LSP = [test_label,test_SD,test_pred]
generator_LSP = [train_LSP,eval_LSP,test_LSP]

In [1370]:
def plotting_with_TS_V(generator_LSP, mode, selected_EP):
    ## make cross plot between prediction and label with highlighted color for subset that meet threashold
    
    fig, axs = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(5,12.5))

    for g, m, ax in zip(generator_LSP, mode, axs): 
        label = g[0]
        SD = g[1]
        pred = g[2].flatten()
#         if len(pred) == len(label):
#             print(len(pred))
        idx = np.abs(pred-label) > 1*SD
        error_pct = np.round(100*np.sum(idx)/len(pred), 1)
        

        ax.plot(pred, label,'b.', alpha=0.1, label=f'Correct ' + str(100 - error_pct) + '%')
        ax.plot(pred[idx], label[idx], 'r.', alpha=0.1, label=f'Incorrect ' + str(error_pct) + '%')
        plt.xticks(fontsize = 12)
        plt.yticks(fontsize = 12)
        ax.plot([0, 1], [0, 1], 'k', alpha=0.5)
        ax.set_title(str(m), fontsize = 24, loc = 'center')
        #text_error = '% error (>2SD) = ' +str(error_pct) + '%'
        #ax.text(0.7, 0.1, text_error, fontsize = 12,
                #horizontalalignment = 'center', transform=ax.transAxes)
        ax.legend(loc='upper left')
        ax.set_ylabel('Experimental Value', fontsize =18)
    axs[2].set_xlabel('CNN Prediction', fontsize = 18)

    plt.tight_layout()
    plt.show
    plt.savefig(exp_folder_path  +'/train_results_.jpeg', dpi=900)
    plt.savefig(exp_folder_path  +'/train_results_.svg', dpi=900)

In [1371]:
plotting_with_TS_V(generator_LSP, ['Train', 'Eval', 'Test'], selected_EP = selectedE)